I decided to go the route of converting images into numpy arrays for training and testing as opposed to using the Keras built-in ImageDataGenerator and FlowFromDirectory (opting instead for a single file of raw image data and discrete file of labels).  In this notebook, I will be building a CNN from scratch an leveraing the different data pipeline methodology

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import PIL

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization

import keras.utils
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#check using system GPU for processing and declaring system/GPU parameters

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

# configure tensorflow before fitting model
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.Session(config=tf_config)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11205786349670335466
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1494830284
locality {
  bus_id: 1
}
incarnation: 2206715240686807994
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
# changing directory to access data (as numpy arrays)
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [4]:
# define functions to load data

def load_array(fname):
    return np.load(open(fname,'rb'))

In [5]:
# load in labels and data (as tensors)

train_labels=load_array('train_labels.npy')
valid_labels=load_array('valid_labels.npy')

In [6]:
train_tensor=load_array('train_dataset.npy')

In [7]:
def Normalize_Input(X):
    minimum=0
    maximum=255
    X-minimum/(maximum-minimum)
    return X  

In [8]:
train_tensor=Normalize_Input(train_tensor)

In [9]:
valid_tensor=load_array('valid_dataset.npy')

In [10]:
valid_tensor=Normalize_Input(valid_tensor)

In [11]:
# feeding the training data through an Image Augmentation process (including resizing and shifting tolerance)

num_classes = 120
batch_size = 12
input_shape = (224, 224, 3)

train_datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1)

validation_datagen = ImageDataGenerator()

# note to self... perhaps the imagedatagenerator parameters I had before were root cause of low accuracy...

train_generator = train_datagen.flow(x=train_tensor, y=train_labels, batch_size=batch_size, shuffle=False, seed=16)
validation_generator = validation_datagen.flow(x=valid_tensor, y=valid_labels, batch_size=batch_size, shuffle=False, seed=16)

In [12]:
wide_model_slow_learn = Sequential()

wide_model_slow_learn.add(BatchNormalization(input_shape=input_shape))
wide_model_slow_learn.add(Conv2D(64, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(BatchNormalization())

wide_model_slow_learn.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(BatchNormalization())

wide_model_slow_learn.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(BatchNormalization())

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(BatchNormalization())

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(BatchNormalization())

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_slow_learn.add(BatchNormalization())

wide_model_slow_learn.add(Dense(2048, activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(Dropout(0.2))
wide_model_slow_learn.add(GlobalAveragePooling2D())

wide_model_slow_learn.add(Dense(num_classes, activation='softmax'))
    
# Compile model

#adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
wide_model_slow_learn.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy']) 
#loss changed to sparse for new label data
print(wide_model_slow_learn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 111, 111, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 54, 54, 64)        256       
__________

In [13]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.bestaugmented.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

In [14]:
wide_model_slow_learn.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=800, epochs=10, callbacks=[checkpointer, early_stopping])

Epoch 1/10
800/800 [==============================] - 171s 214ms/step - loss: 4.8917 - acc: 0.0033 - val_loss: 4.7933 - val_acc: 0.0071

Epoch 00001: val_loss improved from inf to 4.79325, saving model to saved_models/weights.bestaugmented.from_scratch.hdf5
Epoch 2/10
800/800 [==============================] - 160s 200ms/step - loss: 4.8181 - acc: 0.0034 - val_loss: 4.7950 - val_acc: 0.0083

Epoch 00002: val_loss did not improve
Epoch 3/10
800/800 [==============================] - 167s 209ms/step - loss: 4.8077 - acc: 0.0029 - val_loss: 4.7906 - val_acc: 0.0104

Epoch 00003: val_loss improved from 4.79325 to 4.79056, saving model to saved_models/weights.bestaugmented.from_scratch.hdf5
Epoch 4/10
800/800 [==============================] - 160s 200ms/step - loss: 4.7987 - acc: 0.0041 - val_loss: 4.7889 - val_acc: 0.0088

Epoch 00004: val_loss improved from 4.79056 to 4.78889, saving model to saved_models/weights.bestaugmented.from_scratch.hdf5
Epoch 5/10
800/800 [=======================

In [12]:
# increase batch size, less parameters on ImageDataGenerator

batch_size = 20

train_datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

train_generator = train_datagen.flow(x=train_tensor, y=train_labels, batch_size=batch_size, shuffle=False, seed=16)
validation_generator = validation_datagen.flow(x=valid_tensor, y=valid_labels, batch_size=batch_size, shuffle=False, seed=16)

In [17]:
wide_model_slow_learn.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=480, epochs=10, callbacks=[checkpointer, early_stopping])

Epoch 1/10
480/480 [==============================] - 101s 211ms/step - loss: 4.7939 - acc: 0.0047 - val_loss: 4.8078 - val_acc: 0.0042

Epoch 00001: val_loss did not improve
Epoch 2/10
480/480 [==============================] - 101s 210ms/step - loss: 4.7984 - acc: 0.0040 - val_loss: 4.7918 - val_acc: 0.0079

Epoch 00002: val_loss did not improve
Epoch 3/10
480/480 [==============================] - 103s 214ms/step - loss: 4.7941 - acc: 0.0045 - val_loss: 4.7896 - val_acc: 0.0079

Epoch 00003: val_loss did not improve
Epoch 4/10
480/480 [==============================] - 110s 230ms/step - loss: 4.7911 - acc: 0.0061 - val_loss: 4.7926 - val_acc: 0.0054

Epoch 00004: val_loss did not improve
Epoch 5/10
480/480 [==============================] - 104s 216ms/step - loss: 4.7962 - acc: 0.0033 - val_loss: 4.8216 - val_acc: 0.0075

Epoch 00005: val_loss did not improve


In [15]:
# trying new model with increase # of filters and "he_normal" kernel initializer.  "glorot_uniform" is default

new_model = Sequential()

new_model.add(BatchNormalization(input_shape=input_shape))
new_model.add(Conv2D(16, (3, 3), strides=1, kernel_initializer='he_normal', activation='relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))
new_model.add(BatchNormalization())


new_model.add(Conv2D(32, (3, 3), strides=1, kernel_initializer='he_normal', activation='relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))
new_model.add(BatchNormalization())

new_model.add(Conv2D(64, (3, 3), strides=1, kernel_initializer='he_normal', activation='relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))
new_model.add(BatchNormalization())

new_model.add(Conv2D(128, (3, 3), strides=1, kernel_initializer='he_normal', activation='relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))
new_model.add(BatchNormalization())

new_model.add(Conv2D(256, (3, 3), strides=1, kernel_initializer='he_normal', activation='relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))
new_model.add(BatchNormalization())

new_model.add(GlobalAveragePooling2D())

new_model.add(Dense(num_classes, activation='softmax'))

new_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
print(new_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_8 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 111, 111, 16)      64        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 54, 54, 32)        128       
__________

In [16]:
new_model.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=480, epochs=10, callbacks=[checkpointer, early_stopping])

Epoch 1/10
480/480 [==============================] - 91s 189ms/step - loss: 4.8329 - acc: 0.0108 - val_loss: 4.8078 - val_acc: 0.0079

Epoch 00001: val_loss did not improve
Epoch 2/10
480/480 [==============================] - 90s 187ms/step - loss: 4.8269 - acc: 0.0069 - val_loss: 4.7867 - val_acc: 0.0083

Epoch 00002: val_loss improved from 4.78889 to 4.78671, saving model to saved_models/weights.bestaugmented.from_scratch.hdf5
Epoch 3/10
480/480 [==============================] - 89s 185ms/step - loss: 4.8236 - acc: 0.0068 - val_loss: 4.8822 - val_acc: 0.0100

Epoch 00003: val_loss did not improve
Epoch 4/10
480/480 [==============================] - 89s 186ms/step - loss: 4.8192 - acc: 0.0082 - val_loss: 4.7907 - val_acc: 0.0075

Epoch 00004: val_loss did not improve
